In [1]:
#Цикл всегда 80 с переопределением расхода
import prop
import pandas as pd

#Таблица с потоками и блоками
streams = pd.read_excel("streams.xlsx",index_col=0)
blocks  = pd.read_excel("blocks.xlsx" ,index_col=0)

#Параметры нагревающей среды
gas     = 'REFPROP::N2[0.7757]&O2[0.1068]&CO2[0.0193]&H2O[0.0936]&Ar[0.0046]'    #Состав газа
Tin     = 183.6                         #Температура газов на входе
Pgas    = 0.1                           #Давление газов
Ggas    = 509                           #Расход газов
Toutmin = 80                            #Температура газов на выходе
Hgasin  = prop.t_p(Tin,Pgas,gas)["H"]   #Энтальпия газов на входе


#Параметры ОЦР
fluid = "REFPROP::R236ea"
Ppump = 3.3                 #Давление на выходе из насоса
Pcond = 0.243               #Давление в конденсаторе
KPDpump = 0.8               #КПД насоса
KPDturb = 0.9               #КПД турбины
G = 100                     #Расход среды (на 1 шаге)
dTmin = 15                  #Минимальный напор в регенераторе
dTmin_heater = 20           #Напор в нагревателе

#Внос данных в таблицу
streams.loc["IN-HEAT", "T":"Q"] = [Tin,Pgas,Hgasin,prop.h_p(Hgasin,Pgas,gas)["S"],Ggas,1]

#Начало расчёта - поток перед насосом
streams.loc["COND-PUMP", "T":"Q"] = [prop.p_q(Pcond,0,fluid)["T"],Pcond,prop.p_q(Pcond,0,fluid)['H'],prop.p_q(Pcond,0,fluid)['S'],G,0]

#Цикл для сведения с регенератором
for i in range(3):
    #Насос
    from modules import pump
    pump.calc("COND-PUMP","PUMP-REG",Ppump,KPDpump,fluid,streams,blocks)

    #Регенератор
    from modules import regen
    regen.calc("TURB-REG","REG-COND","PUMP-REG","REG-HEAT",dTmin,fluid,streams,blocks)

    #Нагреватель с расчётом расхода
    from modules import heater80
    heater80.calc("IN-HEAT","HEAT-OUT","REG-HEAT","HEAT-TURB",dTmin_heater,Toutmin,gas,fluid,streams,blocks)

    #Турбина
    from modules import turbine
    turbine.calc("HEAT-TURB","TURB-REG",Pcond,KPDturb,fluid,streams,blocks)

    #Регенератор
    from modules import regen
    regen.calc("TURB-REG","REG-COND","PUMP-REG","REG-HEAT",dTmin,fluid,streams,blocks)

    #Конденсатор
    from modules import condenser
    condenser.calc("REG-COND","COND-PUMP",fluid,streams,blocks)

streams

,T,P,H,S,G,Q
IN-HEAT,183.600000,0.100,615.516763,7.334542,509.000000,1.0
HEAT-OUT,80.000000,0.100,503.727572,7.057047,509.000000,1.0
COND-PUMP,29.838015,0.243,237.190329,1.128791,272.834558,0.0
PUMP-REG,31.444901,3.300,239.890113,1.130565,272.834558,0.0
REG-HEAT,63.632373,3.300,281.274379,1.259668,272.834558,0.0
HEAT-TURB,163.600000,3.300,489.828232,1.790765,272.834558,1.0
TURB-REG,90.303055,0.243,446.813183,1.804006,272.834558,1.0
REG-COND,46.444901,0.243,405.428917,1.682728,272.834558,1.0


In [2]:
blocks

,N,Q
PUMP,736.594314,NaN
TURBINE,11735.991910,NaN
REGENERATOR,NaN,11291.057889
CONDENSER,NaN,45901.300581
HEATER,NaN,56900.698177


In [3]:
#Баланс
print(blocks.loc['HEATER','Q']+blocks.loc['PUMP','N']-blocks.loc['CONDENSER','Q']-blocks.loc['TURBINE','N'])

#КПД
KPD =(blocks.loc['TURBINE','N']-blocks.loc['PUMP','N'])/ blocks.loc['HEATER','Q']
print(KPD)

-1.8189894035458565e-12
0.1933086578558544
